<b>The goal of this project is to predict chances of up movement of all S&P 500 stocks. We will start by looping over all members of S&P 500 (T). We then will calculate what is the probability of T to go up after it went down N days (N in range 1-10). Before each each trading day we will calculate the probability of a stock to go up (based on how many consecutive days did it go down till now). Then we will do the same for Sector ETFs. We will use the yahoo finance library to get the stocks closing prices.
</b>

In [1]:
import pandas as pd
import numpy as np
import yfinance as y_f
from pandas_datareader import data as wb

In [2]:
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

In [3]:
df = pd.read_csv('S&P500-Symbols.csv')
list_stock = list(df['Symbol'])

In [4]:
data = y_f.download(list_stock,'2019-1-1')['Adj Close']

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BF.B: None
- BRK.B: No data found, symbol may be delisted


In [5]:
data.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,66.080727,31.599045,153.986130,38.395920,77.750801,70.632858,325.040009,68.979408,135.379837,226.240005,...,45.768890,82.728714,56.769897,36.413403,64.409439,87.240913,101.681244,159.229996,37.717342,84.051025
2019-01-02,64.346909,31.963154,154.435959,38.439735,75.254402,70.680321,309.959991,66.280487,134.976608,224.570007,...,44.793503,84.360565,58.018696,37.078842,64.206734,86.785378,100.269554,156.240005,38.495026,83.009483
2019-01-03,61.976395,29.581665,159.257233,34.610851,72.774887,69.654991,302.290009,63.152431,130.368271,215.699997,...,44.617012,81.184296,57.127892,37.098419,62.005688,84.602448,98.455925,146.880005,38.282089,80.140282
2019-01-04,64.121620,31.530161,155.306335,36.088364,75.119469,70.461960,313.440002,64.954887,135.437469,226.190002,...,45.053604,84.943359,59.234173,38.174862,64.621826,86.804344,101.818497,152.970001,39.457855,83.284599
2019-01-07,65.483200,32.425678,157.584915,36.008041,76.215836,70.965126,314.799988,65.927643,135.907867,229.259995,...,44.858528,87.187141,59.542206,38.859890,63.666126,86.709450,101.857712,155.289993,39.615250,83.785721


In [14]:
#for N 0-10, find all true values which represent a stock growth after consecutive N days of declines.

In [6]:
i_increase_table = dict()
for i in range(11):
    i_increase_table[i] = data.rolling(i).apply(lambda x: np.all(np.diff(x) <= 0)).astype('boolean') & (~data.rolling(i+1).apply(lambda x: np.all(np.diff(x) <= 0)).astype('boolean'))
    
i_increase_table


{0:                 A    AAL    AAP   AAPL   ABBV    ABC   ABMD    ABT    ACN  \
 Date                                                                        
 2018-12-31  False  False  False  False  False  False  False  False  False   
 2019-01-02  False  False  False  False  False  False  False  False  False   
 2019-01-03  False  False  False  False  False  False  False  False  False   
 2019-01-04  False  False  False  False  False  False  False  False  False   
 2019-01-07  False  False  False  False  False  False  False  False  False   
 ...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
 2021-11-26  False  False  False  False  False  False  False  False  False   
 2021-11-29  False  False  False  False  False  False  False  False  False   
 2021-11-30  False  False  False  False  False  False  False  False  False   
 2021-12-01  False  False  False  False  False  False  False  False  False   
 2021-12-02  False  False  False  False  False  False  False 

In [7]:
nominator = [i_increase_table[k].sum() for k in range(11)]
nominator

[A       0
 AAL     0
 AAP     0
 AAPL    0
 ABBV    0
        ..
 YUM     0
 ZBH     0
 ZBRA    0
 ZION    0
 ZTS     0
 Length: 505, dtype: int64,
 A       419
 AAL     346
 AAP     375
 AAPL    408
 ABBV    401
        ... 
 YUM     396
 ZBH     371
 ZBRA    399
 ZION    381
 ZTS     409
 Length: 505, dtype: int64,
 A       187
 AAL     178
 AAP     187
 AAPL    195
 ABBV    186
        ... 
 YUM     178
 ZBH     189
 ZBRA    193
 ZION    191
 ZTS     190
 Length: 505, dtype: int64,
 A       75
 AAL     89
 AAP     90
 AAPL    84
 ABBV    81
         ..
 YUM     92
 ZBH     89
 ZBRA    79
 ZION    88
 ZTS     81
 Length: 505, dtype: int64,
 A       32
 AAL     56
 AAP     49
 AAPL    39
 ABBV    44
         ..
 YUM     40
 ZBH     47
 ZBRA    41
 ZION    43
 ZTS     37
 Length: 505, dtype: int64,
 A       10
 AAL     23
 AAP     22
 AAPL     9
 ABBV    15
         ..
 YUM     16
 ZBH     24
 ZBRA    15
 ZION    22
 ZTS     15
 Length: 505, dtype: int64,
 A        7
 AAL     18
 AAP 

In [8]:
denominator = [sum(i) for i in zip(*nominator)]
denominator

[735,
 735,
 737,
 737,
 735,
 737,
 735,
 735,
 735,
 735,
 737,
 735,
 735,
 737,
 736,
 730,
 735,
 735,
 737,
 737,
 734,
 735,
 737,
 735,
 737,
 735,
 737,
 737,
 717,
 737,
 735,
 735,
 737,
 736,
 737,
 735,
 737,
 734,
 735,
 737,
 736,
 735,
 735,
 735,
 736,
 736,
 735,
 736,
 735,
 737,
 736,
 737,
 737,
 737,
 737,
 734,
 737,
 737,
 735,
 737,
 0,
 737,
 735,
 737,
 735,
 733,
 735,
 735,
 737,
 735,
 0,
 735,
 735,
 737,
 736,
 734,
 736,
 737,
 430,
 735,
 735,
 735,
 735,
 736,
 734,
 735,
 735,
 735,
 737,
 735,
 735,
 737,
 735,
 735,
 736,
 735,
 735,
 735,
 736,
 737,
 737,
 735,
 734,
 735,
 736,
 735,
 736,
 737,
 735,
 737,
 737,
 736,
 735,
 735,
 735,
 735,
 734,
 735,
 735,
 736,
 735,
 635,
 735,
 737,
 737,
 737,
 734,
 736,
 737,
 735,
 737,
 736,
 735,
 737,
 735,
 735,
 737,
 737,
 737,
 732,
 737,
 737,
 681,
 735,
 736,
 737,
 736,
 735,
 737,
 737,
 737,
 735,
 737,
 737,
 735,
 736,
 735,
 736,
 737,
 734,
 735,
 737,
 737,
 735,
 736,
 735,
 736,
 7

In [9]:
probabilities = [nominator[i] / denominator[i] for i in range(len(nominator))]
probabilities

[A       0.0
 AAL     0.0
 AAP     0.0
 AAPL    0.0
 ABBV    0.0
        ... 
 YUM     0.0
 ZBH     0.0
 ZBRA    0.0
 ZION    0.0
 ZTS     0.0
 Length: 505, dtype: float64,
 A       0.570068
 AAL     0.470748
 AAP     0.510204
 AAPL    0.555102
 ABBV    0.545578
           ...   
 YUM     0.538776
 ZBH     0.504762
 ZBRA    0.542857
 ZION    0.518367
 ZTS     0.556463
 Length: 505, dtype: float64,
 A       0.253731
 AAL     0.241520
 AAP     0.253731
 AAPL    0.264586
 ABBV    0.252374
           ...   
 YUM     0.241520
 ZBH     0.256445
 ZBRA    0.261872
 ZION    0.259159
 ZTS     0.257802
 Length: 505, dtype: float64,
 A       0.101764
 AAL     0.120760
 AAP     0.122117
 AAPL    0.113976
 ABBV    0.109905
           ...   
 YUM     0.124830
 ZBH     0.120760
 ZBRA    0.107191
 ZION    0.119403
 ZTS     0.109905
 Length: 505, dtype: float64,
 A       0.043537
 AAL     0.076190
 AAP     0.066667
 AAPL    0.053061
 ABBV    0.059864
           ...   
 YUM     0.054422
 ZBH     0.063946

In [10]:
#calculate the growth rate between a day and a previous day.
#Each postive value reflects by how much a stock grew after x consecutiuve days of decline, marked as negative values.
data_growth = data.pct_change()
data_growth

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,XEL,XLNX,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2018-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-02,-0.026238,0.011523,0.002921,0.001141,-0.032108,0.000672,-0.046394,-0.039126,-0.002978,-0.007382,...,-0.021311,0.019725,0.021998,0.018275,-0.003147,-0.005222,-0.013883,-0.018778,0.020619,-0.012392
2019-01-03,-0.036840,-0.074507,0.031219,-0.099607,-0.032948,-0.014507,-0.024745,-0.047194,-0.034142,-0.039498,...,-0.003940,-0.037651,-0.015354,0.000528,-0.034281,-0.025153,-0.018088,-0.059908,-0.005532,-0.034565
2019-01-04,0.034614,0.065868,-0.024808,0.042689,0.032217,0.011585,0.036885,0.028541,0.038884,0.048632,...,0.009785,0.046303,0.036870,0.029016,0.042192,0.026026,0.034153,0.041462,0.030713,0.039235
2019-01-07,0.021234,0.028402,0.014672,-0.002226,0.014595,0.007141,0.004339,0.014976,0.003473,0.013573,...,-0.004330,0.026415,0.005200,0.017944,-0.014789,-0.001093,0.000385,0.015166,0.003989,0.006017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-11-26,-0.005883,-0.087873,-0.028453,-0.031678,-0.018119,-0.026005,-0.034934,0.006476,-0.024202,-0.009307,...,-0.012052,-0.026516,-0.035129,-0.028677,-0.019194,-0.021122,-0.035695,-0.012207,-0.049192,-0.017251
2021-11-29,0.008877,-0.000563,0.002382,0.021874,0.003261,0.000000,-0.021044,0.017080,0.034559,0.038348,...,0.012816,0.040768,0.005551,-0.022093,0.009459,0.008454,-0.016876,0.027898,0.011532,0.019287
2021-11-30,-0.016426,-0.002818,-0.028820,0.031578,-0.013774,-0.037262,-0.013600,-0.017652,-0.023017,-0.025659,...,-0.028358,-0.019864,-0.028414,0.001027,-0.021729,-0.019242,-0.031579,-0.030336,-0.028193,-0.006755


<b>Sector ETFs</b>

In [15]:
f = open("sectors.txt","r")
stocks = [line.strip() for line in f]
f.close()

wb.DataReader(stocks,"yahoo", start="2019-1-1",end="2021-11-05")["Adj Close"].to_csv("adjcloseSectors.csv")


In [17]:
stocks = pd.read_csv('adjcloseSectors.csv').set_index('Date')
stocks

,XLK,XLE,XLF,XLV,XLRE,XLB,XLY,XLP,XLU,XLI,IYZ
Date,,,,,,,,,,,
2019-01-02,60.107811,49.711246,22.651291,80.982460,27.763735,48.037811,96.892662,46.929024,47.590740,61.585190,24.619350
2019-01-03,57.074314,49.218224,22.142057,79.338097,27.892059,46.675571,94.794731,46.659424,47.581589,59.711193,24.239166
2019-01-04,59.603851,50.892811,22.877609,81.704849,28.176201,48.510803,97.931900,47.654156,48.286167,61.975212,24.980991
2019-01-07,60.136887,51.649368,22.905903,82.018501,28.414520,48.681091,100.146355,47.589081,47.956757,62.479389,25.333361
2019-01-08,60.640850,52.048904,22.924767,82.655334,28.909483,49.191929,101.253609,48.026024,48.551525,63.335537,25.722815
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-01,161.289993,58.459999,40.279999,133.660004,47.930000,85.320000,204.839996,71.360001,67.150002,104.779999,32.139999
2021-11-02,162.710007,57.910000,40.380001,134.500000,48.380001,86.290001,203.460007,71.839996,67.209999,105.129997,32.750000
2021-11-03,163.660004,57.430000,40.529999,135.289993,48.660000,87.190002,207.119995,72.510002,66.959999,104.900002,33.169998


In [18]:
i_increase_table_etfs = dict()
for i in range(11):
    i_increase_table_etfs[i] = stocks.rolling(i).apply(lambda x: np.all(np.diff(x) <= 0)).astype('boolean') & (~stocks.rolling(i+1).apply(lambda x: np.all(np.diff(x) <= 0)).astype('boolean'))

i_increase_table_etfs

{0:               XLK    XLE    XLF    XLV   XLRE    XLB    XLY    XLP    XLU  \
 Date                                                                        
 2019-01-02  False  False  False  False  False  False  False  False  False   
 2019-01-03  False  False  False  False  False  False  False  False  False   
 2019-01-04  False  False  False  False  False  False  False  False  False   
 2019-01-07  False  False  False  False  False  False  False  False  False   
 2019-01-08  False  False  False  False  False  False  False  False  False   
 ...           ...    ...    ...    ...    ...    ...    ...    ...    ...   
 2021-11-01  False  False  False  False  False  False  False  False  False   
 2021-11-02  False  False  False  False  False  False  False  False  False   
 2021-11-03  False  False  False  False  False  False  False  False  False   
 2021-11-04  False  False  False  False  False  False  False  False  False   
 2021-11-05  False  False  False  False  False  False  False 

In [19]:
nominator = [i_increase_table_etfs[k].sum() for k in range(11)]
nominator

[XLK     0
 XLE     0
 XLF     0
 XLV     0
 XLRE    0
 XLB     0
 XLY     0
 XLP     0
 XLU     0
 XLI     0
 IYZ     0
 dtype: int64,
 XLK     409
 XLE     366
 XLF     395
 XLV     391
 XLRE    412
 XLB     395
 XLY     417
 XLP     394
 XLU     395
 XLI     397
 IYZ     372
 dtype: int64,
 XLK     186
 XLE     180
 XLF     187
 XLV     193
 XLRE    184
 XLB     191
 XLY     175
 XLP     190
 XLU     179
 XLI     180
 IYZ     185
 dtype: int64,
 XLK     76
 XLE     80
 XLF     71
 XLV     76
 XLRE    75
 XLB     79
 XLY     77
 XLP     83
 XLU     79
 XLI     80
 IYZ     87
 dtype: int64,
 XLK     28
 XLE     45
 XLF     35
 XLV     33
 XLRE    34
 XLB     33
 XLY     34
 XLP     33
 XLU     36
 XLI     34
 IYZ     48
 dtype: int64,
 XLK      9
 XLE     21
 XLF     16
 XLV     12
 XLRE     9
 XLB     13
 XLY      8
 XLP     13
 XLU     12
 XLI     14
 IYZ     16
 dtype: int64,
 XLK      5
 XLE     14
 XLF      9
 XLV      8
 XLRE     3
 XLB      5
 XLY      4
 XLP      4
 XLU      4

In [20]:
denominator = [sum(i) for i in zip(*nominator)]
denominator

[717, 717, 717, 717, 718, 717, 717, 717, 712, 715, 717]

In [21]:
probabilities = [nominator[i] / denominator[i] for i in range(len(nominator))]
probabilities

[XLK     0.0
 XLE     0.0
 XLF     0.0
 XLV     0.0
 XLRE    0.0
 XLB     0.0
 XLY     0.0
 XLP     0.0
 XLU     0.0
 XLI     0.0
 IYZ     0.0
 dtype: float64,
 XLK     0.570432
 XLE     0.510460
 XLF     0.550907
 XLV     0.545328
 XLRE    0.574616
 XLB     0.550907
 XLY     0.581590
 XLP     0.549512
 XLU     0.550907
 XLI     0.553696
 IYZ     0.518828
 dtype: float64,
 XLK     0.259414
 XLE     0.251046
 XLF     0.260809
 XLV     0.269177
 XLRE    0.256625
 XLB     0.266388
 XLY     0.244073
 XLP     0.264993
 XLU     0.249651
 XLI     0.251046
 IYZ     0.258020
 dtype: float64,
 XLK     0.105997
 XLE     0.111576
 XLF     0.099024
 XLV     0.105997
 XLRE    0.104603
 XLB     0.110181
 XLY     0.107392
 XLP     0.115760
 XLU     0.110181
 XLI     0.111576
 IYZ     0.121339
 dtype: float64,
 XLK     0.038997
 XLE     0.062674
 XLF     0.048747
 XLV     0.045961
 XLRE    0.047354
 XLB     0.045961
 XLY     0.047354
 XLP     0.045961
 XLU     0.050139
 XLI     0.047354
 IYZ     0.0668

In [23]:
stock_growth_etfs = stocks.pct_change()
stock_growth_etfs

,XLK,XLE,XLF,XLV,XLRE,XLB,XLY,XLP,XLU,XLI,IYZ
Date,,,,,,,,,,,
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-03,-0.050468,-0.009918,-0.022481,-0.020305,0.004622,-0.028358,-0.021652,-0.005745,-0.000192,-0.030429,-0.015442
2019-01-04,0.044320,0.034024,0.033220,0.029831,0.010187,0.039319,0.033094,0.021319,0.014808,0.037916,0.030604
2019-01-07,0.008943,0.014866,0.001237,0.003839,0.008458,0.003510,0.022612,-0.001366,-0.006822,0.008135,0.014105
2019-01-08,0.008380,0.007736,0.000824,0.007765,0.017419,0.010494,0.011056,0.009182,0.012402,0.013703,0.015373
...,...,...,...,...,...,...,...,...,...,...,...
2021-11-01,-0.001486,0.017226,0.000497,-0.001196,0.002300,0.002350,0.018345,0.001544,0.003587,0.002775,0.010692
2021-11-02,0.008804,-0.009408,0.002483,0.006285,0.009389,0.011369,-0.006737,0.006726,0.000893,0.003340,0.018979
2021-11-03,0.005839,-0.008289,0.003715,0.005874,0.005787,0.010430,0.017989,0.009326,-0.003720,-0.002188,0.012824
